In [1]:
import pandas as pd
import json


In [2]:
DB_DATA_PATH = "data/HungThinhDB10K.csv"

db_data = pd.read_csv(DB_DATA_PATH)


In [3]:
with open('data/intent_alias_data.json', 'r', encoding="utf-8") as fp:
    alias_file = json.load(fp)
    fp.close()


In [14]:

with open('data/create_csv_dict.json', 'r', encoding="utf-8") as fp:
    create_csv_dict = json.load(fp)
    fp.close()

HOUSE = create_csv_dict["HOUSE"]
CITY = create_csv_dict["CITY"]
DISTRICT = create_csv_dict["DISTRICT"]
WARD = create_csv_dict["WARD"]
STREET = create_csv_dict["STREET"]
HOUSE_NUMBER = create_csv_dict["HOUSE_NUMBER"]


real_estate_sub_type


In [28]:
def preprocess(old_data, old_column_name, new_column_name):
    temp_data = pd.DataFrame(old_data)
    temp_data = temp_data.rename(columns={old_column_name: new_column_name})
    temp_data = temp_data.dropna()
    temp_data = temp_data.drop_duplicates()

    for x in temp_data.index:
        if "http" in temp_data.loc[x, new_column_name]:
            # print(x, temp_data.loc[x, new_column_name])
            temp_data = temp_data.drop(x)

        # if "/" in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
            # temp_data = temp_data.drop(x)

        # if "." in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
            # temp_data = temp_data.drop(x)

        # if "<" in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
        #     temp_data = temp_data.drop(x)

        # if ">" in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
        #     temp_data = temp_data.drop(x)

        # if "01" in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
        #     temp_data = temp_data.drop(x)

    return temp_data


In [29]:
def add_alias_to_csv(file_csv_path, new_alias_column_name, alias_list):
    temp_data = pd.read_csv(file_csv_path)
    new_column = pd.DataFrame({new_alias_column_name: alias_list})
    temp_data = temp_data.merge(
        new_column, how="left", left_index=True, right_index=True)
    temp_data.to_csv(file_csv_path, index=False)


In [30]:
def create_alias_column(file_csv_path, new_alias_column_name, alias_file, alias_data):
    alias_list = alias_file[alias_data]
    add_alias_to_csv(file_csv_path, new_alias_column_name, alias_list)


In [31]:
def create_csv_file(db_data, column_name, new_column_name, file_name):
    temp_data_old = db_data[[column_name]]
    new_data = preprocess(temp_data_old, column_name, new_column_name)
    new_data.to_csv(file_name)


In [32]:
# implement house
create_csv_file(db_data, HOUSE["column_name"],
                HOUSE["new_column_name"], HOUSE["csv_file_path"])
create_alias_column(HOUSE["csv_file_path"], HOUSE["new_alias_column_name"],
                    alias_file, HOUSE["alias_list"])


In [13]:
# implement city
create_csv_file(db_data, CITY["column_name"],
                CITY["new_column_name"], CITY["csv_file_path"])
create_alias_column(CITY["csv_file_path"], CITY["new_alias_column_name"],
                    alias_file, CITY["alias_list"])

In [143]:
# implement district
create_csv_file(db_data, DISTRICT["column_name"],
                DISTRICT["new_column_name"], DISTRICT["csv_file_path"])
create_alias_column(DISTRICT["csv_file_path"], DISTRICT["new_alias_column_name"],
                    alias_file, DISTRICT["alias_list"])

In [145]:
# implement ward
create_csv_file(db_data, WARD["column_name"],
                WARD["new_column_name"], WARD["csv_file_path"])
create_alias_column(WARD["csv_file_path"], WARD["new_alias_column_name"],
                    alias_file, WARD["alias_list"])


In [146]:
# implement street
create_csv_file(db_data, STREET["column_name"],
                STREET["new_column_name"], STREET["csv_file_path"])
create_alias_column(STREET["csv_file_path"], STREET["new_alias_column_name"],
                    alias_file, STREET["alias_list"])


In [147]:
# implement house_number
create_csv_file(db_data, HOUSE_NUMBER["column_name"],
                HOUSE_NUMBER["new_column_name"], HOUSE_NUMBER["csv_file_path"])
create_alias_column(HOUSE_NUMBER["csv_file_path"], HOUSE_NUMBER["new_alias_column_name"],
                    alias_file, HOUSE_NUMBER["alias_list"])
